In [1]:
# Example that prints the current virtual amounts of assets in the 0.3% USDC/ETH pool
# using liquidity data from the Uniswap v3 subgraph.
# 这个文件其实和subgraph-liquidity-positions.ipynb里的内容大部分是一样的

import json, math
import urllib.request

# Look at the USDC/ETH 0.3% pool
POOL_ID = '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8'

# If you want to test with a smaller pool:
# 0.05% GUSD/DAI pool (small pool)
#POOL_ID = "0x7cf12cef5ce9e5e068ebdef470ff8295e26c47b9"

URL = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"

In [2]:
# GraphQL query to get the pool information
query = """query pools($pool_id: ID!) {
  pools (where: {id: $pool_id}) {
    tick
    liquidity
    feeTier
    token0 {
      symbol
      decimals
    }
    token1 {
      symbol
      decimals
    }
  }
}"""

block_query = """query {
  _meta {
    block {
      number
    }
  }
}"""

In [3]:
TICK_BASE = 1.0001

# Convert Uniswap v3 tick to a price (i.e. the ratio between the amounts of tokens: token1/token0)
def tick_to_price(tick):
    return TICK_BASE ** tick

'''
 fee_tier_to_tick_spacing是fee tier和tick spacing之间的对应关系
 tick spacing 代表价格的精度，tick spacing越大精度越小
 详细解释可以看formulas/tickSpacing-and-fee.jpg 和 formulas/tick-spacing.md
'''
# Not all ticks can be initialized. Tick spacing is determined by the pool's fee tier.
def fee_tier_to_tick_spacing(fee_tier):
    return {
        500: 10,
        3000: 60,
        10000: 200
    }.get(fee_tier, 60)
     

In [4]:
# Query the subgraph
req = urllib.request.Request(URL)
req.add_header('Content-Type', 'application/json; charset=utf-8')
jsondata = {"query" : block_query}
jsondataasbytes = json.dumps(jsondata).encode('utf-8')
req.add_header('Content-Length', len(jsondataasbytes))
response = urllib.request.urlopen(req, jsondataasbytes)
assert response.status == 200, "Bad response"

In [5]:

obj = json.load(response)
blocknum = obj['data']['_meta']['block']['number']
print("Ethereum block height {}".format(blocknum))

Ethereum block height 16202793


In [9]:
obj


{'data': {'pools': [{'tick': '205573',
    'liquidity': '12855162560923118222',
    'feeTier': '3000',
    'token0': {'symbol': 'USDC', 'decimals': '6'},
    'token1': {'symbol': 'WETH', 'decimals': '18'}}]}}

In [10]:

# Extract liquidity from the response
L = int(pool["liquidity"])
tick = int(pool["tick"])
ft = int(pool["feeTier"])
tick_spacing = fee_tier_to_tick_spacing(ft)

print("L={}".format(L))
print("tick={}".format(tick))
print("tick spacing={} for fee-tier {}%".format(tick_spacing, ft/1e4))

L=12855162560923118222
tick=205573
tick spacing=60 for fee-tier 0.3%
